In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import pandas as pd
import requests
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import time

from bs4 import BeautifulSoup

In [2]:
data = pd.read_csv('join_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   tempat               909 non-null    object 
 1   tempat_halaltrip     907 non-null    object 
 2   jenis                592 non-null    object 
 3   img                  654 non-null    object 
 4   link_maps            909 non-null    object 
 5   coo1                 909 non-null    object 
 6   coo2                 909 non-null    object 
 7   coordinat            909 non-null    object 
 8   id_tempat            909 non-null    object 
 9   id_tempat_halaltrip  412 non-null    object 
 10  Unnamed:0            412 non-null    float64
 11  city                 445 non-null    object 
 12  description          436 non-null    object 
 13  ticket               416 non-null    object 
 14  opening              417 non-null    object 
 15  recommended          403 non-null    obj

In [3]:
print(data.isnull().sum())
nan_values = data[data['city'].isna()]
nan_values = nan_values.reset_index(drop=True)
nan_values.head()

tempat                   0
tempat_halaltrip         2
jenis                  317
img                    255
link_maps                0
coo1                     0
coo2                     0
coordinat                0
id_tempat                0
id_tempat_halaltrip    497
Unnamed:0              497
city                   464
description            473
ticket                 493
opening                492
recommended            506
suitable               513
mustSee                517
prayerFacilities       519
halalFood              507
link_site              459
dtype: int64


,tempat,tempat_halaltrip,jenis,img,link_maps,coo1,coo2,coordinat,id_tempat,id_tempat_halaltrip,...,city,description,ticket,opening,recommended,suitable,mustSee,prayerFacilities,halalFood,link_site
0,Akita Museum of Art,Akita Museum of Art,Art museum\n,NaN,https://www.google.com/maps/search/Akita+Museu...,39.7173708,140.1208814,"39.7173708,140.1208814",64bb2bbc8e9cfbfd431c65f94147156f4cf776b3f6b346...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.halaltrip.com/attraction-details/1...
1,Alameda del Boulevard,Alameda del Boulevard,NaN,NaN,https://www.google.com/maps/place/Alameda+del+...,-6.9555003,107.7044087,"-6.9555003,107.7044087",308c3ad07da5de57f5ce2fb96ce57db4e314fd502f206c...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alhambra,Alhambra,NaN,NaN,"https://www.google.com/maps/search/Alhambra,+G...",37.1757807,-3.5989404,"37.1757807,-3.5989404",fbc91f2bedd66acc635546c23248f62ef310f167999779...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Amer Fort,Amer Fort,NaN,NaN,https://www.google.com/maps/place/Amber+Palace...,-6.9555003,107.7044087,"-6.9555003,107.7044087",eb0899f0a3785e6bd4462f0649e0a24ba66b9c93588835...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Anek Kuson Sala,Anek Kuson Sala,NaN,NaN,https://www.google.com/maps/search/Anek+Kuson+...,7.0414763,97.9254975,"7.0414763,97.9254975",0caea5923eb82b6c4ee986145ca46163b543f052985956...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
len(nan_values)

464

In [5]:
driver = webdriver.Chrome('/home/ade/Documents/TA2022/chromedriver')

/tmp/ipykernel_1870/1788907942.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/home/ade/Documents/TA2022/chromedriver')


In [6]:
name = []
description = []
ticket = []
opening = []
recommended = []
suitable = []
mustSee = []
prayerFacilities = []
halalFood = []
link_site = []
city = []
link_city=[]


restaurants=[]
add_restaurants=[]
distance_restaurants = []
add_city_resto=[]

mosque=[]
add_mosque=[]
distance_mosque =[]
add_city_mosque=[]

halalDished=[]
add_halalDished=[]
distance_halalDished =[]
add_city_dished=[]


n_key = nan_values.tempat_halaltrip
key = n_key[0]
key

'Akita Museum of Art'

In [7]:
def scrape():        
    try:
        name.append(key)
    except:
        name.append('NaN')
    link_site.append(str(driver.current_url))
    try:
        title_deskripsi = driver.find_element(By.XPATH,'//*[@id="about"]/div/h3[1]').text
        deskripsi = driver.find_element(By.XPATH,'//*[@id="about"]/div/p[1]').text
        description.append(title_deskripsi+" { "+deskripsi+" }")
    except:
        description.append("NaN")
    try:
        title_ticket = driver.find_element(By.XPATH,'//*[@id="about"]/div/h3[2]').text
        ticket_ = driver.find_element(By.XPATH,'//*[@id="about"]/div/p[2]').text
        ticket.append(title_ticket+" { "+ticket_+" }")
    except:
        ticket.append("NaN")
    try:
        title_opening = driver.find_element(By.XPATH,'//*[@id="about"]/div/h3[3]').text
        opening_ = driver.find_element(By.XPATH,'//*[@id="about"]/div/p[3]').text
        opening.append(title_opening+" { "+opening_+" }")
    except:
        opening.append("NaN")
    try:
        title_recommended = driver.find_element(By.XPATH,'//*[@id="about"]/div/h3[4]').text
        recommended_ = driver.find_element(By.XPATH,'//*[@id="about"]/div/p[4]').text
        recommended.append(title_recommended+" { "+recommended_+" }")
    except:
        recommended.append("NaN")
    try:
        title_suitable = driver.find_element(By.XPATH,'//*[@id="about"]/div/h3[5]').text
        suitable_ = driver.find_element(By.XPATH,'//*[@id="about"]/div/p[5]').text
        suitable.append(title_suitable+" { "+suitable_+" }")
    except:
        suitable.append("NaN")
    try:
        title_mustSee = driver.find_element(By.XPATH,'//*[@id="about"]/div/h3[6]').text
        mustSee_ = driver.find_element(By.XPATH,'//*[@id="about"]/div/p[6]').text
        mustSee.append(title_mustSee+" { "+mustSee_+" }")
    except:
        mustSee.append("NaN")
    try:
        title_prayerFacilities = driver.find_element(By.XPATH,'//*[@id="about"]/div/h3[7]').text
        prayerFacilities_ = driver.find_element(By.XPATH,'//*[@id="about"]/div/p[7]').text
        prayerFacilities.append(title_prayerFacilities+" { "+prayerFacilities_+" }")
    except:
        prayerFacilities.append("NaN")
    try:
        title_halalFood = driver.find_element(By.XPATH,'//*[@id="about"]/div/h3[4]').text
        halalFood_ = driver.find_element(By.XPATH,'//*[@id="about"]/div/p[4]').text
        halalFood.append(title_halalFood+" { "+halalFood_+" }")
    except:
        halalFood.append("NaN")
    try:
        city_ = driver.find_element(By.XPATH,'/html/body/section[5]/div[1]/div/div[1]/div/div/p[2]/a').text
        city.append(city_)
    except:
        city.append("NaN")
    try:
        elems = driver.find_element(By.XPATH,"/html/body/section[5]/div[1]/div/div[1]/div/div/p[2]/a")
        if len(elems)==0:
            link_city.append('NaN')
        else:
            for src in elems:
                link_city.append(src.get_attribute('href'))
    except:
        link_city.append("NaN")                
    # resto
    select = driver.find_element(By.XPATH,'/html/body/section[4]/div/div/div/div/ul/li[3]/a')
    select.click()
    response = BeautifulSoup(driver.page_source, 'html.parser')
    categories_table1 = response.find("div", {"id": "restarnt"})
    for each in categories_table1.findAll("li"):
        try:
            add_city_resto_ = driver.find_element(By.XPATH,'/html/body/section[5]/div[1]/div/div[1]/div/div/p[2]/a').text
            add_city_resto.append(add_city_resto_)
        except:
            add_city_resto.append('NaN')
        try:
            restaurants.append(each.h3.text)
        except:
            restaurants.append("NaN")
        try:
            add_restaurants.append(each.p.text)
        except:
            add_restaurants.append("NaN")
        try:
            distance_restaurants.append(each.span.text)
        except:
            distance_restaurants.append("NaN")        
    # halal dished
    select = driver.find_element(By.XPATH,'//*[@id="nearby"]/div/ul/li[2]/a')
    select.click()
    response = BeautifulSoup(driver.page_source, 'html.parser')
    categories_table2 = response.find("div", {"id": "Foodspt"})
    for each in categories_table2.findAll("li"):
        try:
            add_city_dished_ = driver.find_element(By.XPATH,'/html/body/section[5]/div[1]/div/div[1]/div/div/p[2]/a').text
            add_city_dished.append(add_city_dished_)
        except:
            add_city_dished.append('NaN')
        try:
            halalDished.append(each.h3.text)
        except:
            halalDished.append("NaN")
        try:
            add_halalDished.append(each.p.text)
        except:
            add_halalDished.append("NaN")
        try:
            distance_halalDished.append(each.span.text)
        except:
            distance_halalDished.append("NaN")            
    # mosque
    select = driver.find_element(By.XPATH,'//*[@id="nearby"]/div/ul/li[3]/a')
    select.click()
    response = BeautifulSoup(driver.page_source, 'html.parser')
    categories_table3 = response.find("div", {"id": "mosque"})
    for each in categories_table3.findAll("li"):
        try:
            add_city_mosque_ = driver.find_element(By.XPATH,'/html/body/section[5]/div[1]/div/div[1]/div/div/p[2]/a').text
            add_city_mosque.append(add_city_mosque_)
        except:
            add_city_mosque.append('NaN')
        try:
            mosque.append(each.h3.text)
        except:
            mosque.append("NaN")
        try:
            add_mosque.append(each.p.text)
        except:
            add_mosque.append("NaN")
        try:
            distance_mosque.append(each.span.text)
        except:
            distance_mosque.append("NaN")

In [8]:
def cari():
    url ="https://www.google.com"
    driver.get(url)
    
    Place = driver.find_element("name","q")
    Place.send_keys(key,' site:halaltrip.com')
    Place.submit()
    
    try:
        select = driver.find_element(By.XPATH,'//*[@id="rso"]/div[2]/div/div/div[1]/div/a/h3')
        select.click()
    except:
        select = driver.find_element(By.XPATH,'//*[@id="rso"]/div[1]/div/div/div/div[1]/div/a/h3')
        select.click()

In [9]:
cari()

In [11]:
scrape()

In [36]:
i = 14
i

14

## Looping cari & scrape by index

In [31]:
# key = n_key[2]
print(key)
cari()
# if popup close popup

Alameda del Boulevard


In [30]:
scrape()

In [70]:
# i +=1
# key = n_key[i]
# print(key)

# Versi Automatis

In [38]:
i=461
n = n_key[i:]
n

461    Ōhori Park
462           NaN
463           NaN
Name: tempat_halaltrip, dtype: object

In [39]:
for i in n:
    key = i
    print(key)
    cari()
#     # if popup close popup
    scrape()

Ōhori Park
nan


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/section[4]/div/div/div/div/ul/li[3]/a"}
  (Session info: chrome=105.0.5195.52)
Stacktrace:
#0 0x55ba953b7693 <unknown>
#1 0x55ba951b0b0a <unknown>
#2 0x55ba951e95f7 <unknown>
#3 0x55ba951e97c1 <unknown>
#4 0x55ba9521c804 <unknown>
#5 0x55ba9520694d <unknown>
#6 0x55ba9521a4b0 <unknown>
#7 0x55ba95206743 <unknown>
#8 0x55ba951dc533 <unknown>
#9 0x55ba951dd715 <unknown>
#10 0x55ba954077bd <unknown>
#11 0x55ba9540abf9 <unknown>
#12 0x55ba953ecf2e <unknown>
#13 0x55ba9540b9b3 <unknown>
#14 0x55ba953e0e4f <unknown>
#15 0x55ba9542aea8 <unknown>
#16 0x55ba9542b052 <unknown>
#17 0x55ba9544571f <unknown>
#18 0x7f7040187609 <unknown>


In [40]:
import numpy as np
df = pd.DataFrame(np.column_stack([name,city,description,ticket,opening,recommended,suitable,mustSee,prayerFacilities,halalFood,link_site,link_city]),
                  columns=['name','city','description','ticket','opening','recommended','suitable','mustSee','prayerFacilities','halalFood','link_site','link_city'])
df.tail(5)

,name,city,description,ticket,opening,recommended,suitable,mustSee,prayerFacilities,halalFood,link_site,link_city
61,Ölüdeniz Butterfly Valley,"Oludeniz, Turkey",Description { Located in the vicinity of Fethi...,Ticket Price { Not applicable },Opening Hour { Daytime visits are recommended },Recommended Visit Duration { Around 2 – 3 hours },"Suitable For { Adults, Families, Senior Guests...",Must See { The Jersey Tiger butterfly },Prayer Facilities { Though designated prayer r...,Recommended Visit Duration { Around 2 – 3 hours },https://www.halaltrip.com/attraction-details/1...,NaN
62,Ölüdeniz Lagoon and Beach,"Mugla, Turkey",Description { The Blue Lagoon in Ölüdeniz is a...,Ticket Price { Around 5 TL per person },Opening Hour { The beach area is open from mor...,Recommended Visit Duration { Around 2 – 3 hours },"Suitable For { Adults, Families, Senior Guests...",Must See { The beach area and mountains },Prayer Facilities { Though designated prayer r...,Recommended Visit Duration { Around 2 – 3 hours },https://www.halaltrip.com/attraction-details/1...,NaN
63,Ölüdeniz Turkish baths,"Oludeniz, Turkey",Description { Ölüdeniz offers a number of fant...,Ticket Price { Prices vary depending on the se...,Opening Hour { Varies according to the treatme...,Recommended Visit Duration { Around 2 – 3 hours },"Suitable For { Adults, Families, Senior Guests...",Must See { Bath and massage },Prayer Facilities { Access to prayer room faci...,Recommended Visit Duration { Around 2 – 3 hours },https://www.halaltrip.com/attraction-details/1...,NaN
64,Ōhori Park,"Fukuoka, Japan",Description { Ōhori Park is a beautiful park l...,Ticket Price { ¥240 for adults and ¥120 for ch...,Opening Hour { Open daily (except on Mondays) ...,Recommended Visit Duration { Around 1 hour },"Suitable For { Adults, Families, Young Adults,...",Must See { The rock garden and the tea rooms },Prayer Facilities { Though designated prayer f...,Recommended Visit Duration { Around 1 hour },https://www.halaltrip.com/attraction-details/3...,NaN
65,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.halaltrip.com/other/blog/wisata-di...,NaN


In [41]:
df_1 = pd.DataFrame(np.column_stack([restaurants,add_restaurants,distance_restaurants,add_city_resto]),
                  columns=['restaurants','add_restaurants','distance_restaurants','add_city_resto'])
df_2 = pd.DataFrame(np.column_stack([mosque,add_mosque,distance_mosque,add_city_mosque]),
                  columns=['mosque','add_mosque','distance_mosque','add_city_mosque'])
df_3 = pd.DataFrame(np.column_stack([halalDished,add_halalDished,distance_halalDished,add_city_dished]),
                  columns=['halalDished','add_halalDished','distance_halalDished','add_city_dished'])

In [42]:
df_1.tail()

,restaurants,add_restaurants,distance_restaurants,add_city_resto
219,極味や西新店 키와미야 니시진점 (Halal Motsunabe),"4 Chome-9-3 Nishijin, Sawara Ward, Fukuoka, 81...",1.55KM,"Fukuoka, Japan"
220,Zaeka,"2F 1-3-4maihama chuo-ku fukuokashi, Fukuoka 81...",1.62KM,"Fukuoka, Japan"
221,Ayu Werdhi,"\n6th fl., 2-3-2 Daimyo, Chuo-ku, Fukuoka\n",1.66KM,"Fukuoka, Japan"
222,Hyder,"2-9-10, Kego, Chuo-ku, Fukuoka-shi, Fukuoka 81...",1.69KM,"Fukuoka, Japan"
223,Kebabooz,"\n 2-1-17, Fukuoka-shi, Fukuoka, Japan\n",1.94KM,"Fukuoka, Japan"


In [43]:
df_2.tail()

,mosque,add_mosque,distance_mosque,add_city_mosque
214,Ovacık Cami,Ovacık Mh. 48340 Ölüdeniz/Muğla Turkey,0.80KM,"Oludeniz, Turkey"
215,Ölüdeniz Ovacık Cami,Ovacık Mh. 48340 Ölüdeniz/Muğla Turkey,0.81KM,"Oludeniz, Turkey"
216,Hisarönü Cami,Hisarönü Mh. 48340 Ölüdeniz/Muğla Turkey,1.12KM,"Oludeniz, Turkey"
217,YeniHamidiye Mosque,Cumhuriyet Mh. 48870 Fethiye/Muğla Turkey,5.38KM,"Oludeniz, Turkey"
218,Fukuoka Mosque,"3 Chome-2-18 Hakozaki Higashi Ward, Fukuoka, F...",5.96KM,"Fukuoka, Japan"


In [44]:
df_3.tail()

,halalDished,add_halalDished,distance_halalDished,add_city_dished
133,NaN,NaN,NaN,"Mugla, Turkey"
134,NaN,NaN,NaN,"Oludeniz, Turkey"
135,halal Motsunabe,"4 Chome-9-3 Nishijin, Sawara Ward, Fukuoka, 81...",1.55KM,"Fukuoka, Japan"
136,Tempura,"9F JR Hakata City Amu Plaza Hakata, Fukuoka 81...",4.26KM,"Fukuoka, Japan"
137,Chicken Biryani,"6-11-37 Hakozaki, Higashi-ku, Fukuoka",5.76KM,"Fukuoka, Japan"


In [159]:
# cut
# name,city,description,ticket,opening,recommended,suitable,mustSee,prayerFacilities,halalFood,link_site,link_city

In [45]:
df.to_csv('halaltrip_objek-450.csv')
df_1.to_csv('halaltrip_resto-450.csv')
df_2.to_csv('halaltrip_dished-450.csv')
df_3.to_csv('halaltrip_mosque-450.csv')

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              66 non-null     object
 1   city              66 non-null     object
 2   description       66 non-null     object
 3   ticket            66 non-null     object
 4   opening           66 non-null     object
 5   recommended       66 non-null     object
 6   suitable          66 non-null     object
 7   mustSee           66 non-null     object
 8   prayerFacilities  66 non-null     object
 9   halalFood         66 non-null     object
 10  link_site         66 non-null     object
 11  link_city         66 non-null     object
dtypes: object(12)
memory usage: 6.3+ KB


In [47]:
df_1['restaurants'].value_counts()

NaN                               16
Seashell                           3
McKebab                            3
Pizza Hut                          3
Arian restaurant                   3
                                  ..
Orchard Parade Staff Cafeteria     1
Khyber                             1
Delhi Darbar Restaurant            1
Bade Miyan                         1
Kebabooz                           1
Name: restaurants, Length: 188, dtype: int64

In [48]:
df_1['add_city_resto'].value_counts()

Singapore, Singapore         11
Oludeniz, Turkey             10
Newyork, United Kingdom      10
NaN                           9
Yiwu, China                   6
Athens, Greece                5
Marseille, France             5
Dehiwala, Sri Lanka           5
Phi Phi Don, Thailand         5
Dunedin, New Zealand          5
Unawatuna, Sri Lanka          5
Kota Bharu, Malaysia          5
Venice, Italy                 5
Ho Chi Minh City, Vietnam     5
Wadduwa, Sri Lanka            5
Tokyo, Japan                  5
Shenzhen, China               5
Melbourne, Australia          5
Beijing, China                5
Zhuhai, China                 5
Hobart, Australia             5
Mugla, Turkey                 5
London, United Kingdom        5
Bintan, Indonesia             5
New York, USA                 5
Cebu City, Philippines        5
Taipei, Taiwan                5
Bandung, Indonesia            5
Phuket Town, Thailand         5
Phra Khanong, Thailand        5
Kusadası, Turkey              5
Killarne

In [49]:
df.tail(3)

,name,city,description,ticket,opening,recommended,suitable,mustSee,prayerFacilities,halalFood,link_site,link_city
63,Ölüdeniz Turkish baths,"Oludeniz, Turkey",Description { Ölüdeniz offers a number of fant...,Ticket Price { Prices vary depending on the se...,Opening Hour { Varies according to the treatme...,Recommended Visit Duration { Around 2 – 3 hours },"Suitable For { Adults, Families, Senior Guests...",Must See { Bath and massage },Prayer Facilities { Access to prayer room faci...,Recommended Visit Duration { Around 2 – 3 hours },https://www.halaltrip.com/attraction-details/1...,NaN
64,Ōhori Park,"Fukuoka, Japan",Description { Ōhori Park is a beautiful park l...,Ticket Price { ¥240 for adults and ¥120 for ch...,Opening Hour { Open daily (except on Mondays) ...,Recommended Visit Duration { Around 1 hour },"Suitable For { Adults, Families, Young Adults,...",Must See { The rock garden and the tea rooms },Prayer Facilities { Though designated prayer f...,Recommended Visit Duration { Around 1 hour },https://www.halaltrip.com/attraction-details/3...,NaN
65,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.halaltrip.com/other/blog/wisata-di...,NaN
